In [ ]:
import os
import subprocess
from tqdm import tqdm  

# Define paths to your atlas files and MNI space file
atlas_paths = {
    'network': '/media/mm/Mahdi-2022/Atlas/Networks_Atlas/networks_ROIs.nii',
    'visual': '/media/mm/Mahdi-2022/Atlas/Visual_Atlas/visual_ROIs.nii',
    'motor': '/media/mm/Mahdi-2022/Atlas/Motor_Atlas/motor_ROIs.nii'
}
mni_path = '/media/mm/Mahdi-2022/Atlas/bet_mni.nii'

# Define which registrations to perform ### TOGGLE as NEEDED
register_network = False
register_visual = True
register_motor = False

def mirtk_registration(patient_folders):
    base_path = '/media/mm/Mahdi-2022/Laxman_PD_Project/MAP_complete'  # Base path for patient data
    total_patients = len(patient_folders)
    patient_progress = tqdm(patient_folders, desc='Overall Patient Progress')

    for patient_id in patient_progress:
        patient_dir = f'{base_path}/{patient_id}/dti_outputs'  # Construct the full path to dti_outputs
        if not os.path.exists(patient_dir):
            print(f"Directory does not exist: {patient_dir}")
            continue  # Skip this patient if the directory does not exist
        os.chdir(patient_dir)
        
        print(f"Starting registration for {patient_id}")
        b0_nii_path = 'b0.nii'
        b0_exists = os.path.exists(b0_nii_path)

        total_steps = 3 + sum([register_network, register_visual, register_motor])  # Initial steps + conditional atlas registrations
        progress = tqdm(total=total_steps, desc=f'Total Progress for {patient_id}', leave=False)

        # Check if b0.nii already exists; if not, convert b0.mif to b0.nii
        if not b0_exists:
            print(f"Converting b0.mif to b0.nii for {patient_id}")
            subprocess.run(['mrconvert', 'b0.mif', b0_nii_path])
        progress.update()  # Update progress after conversion or acknowledge existing file

        # BET extraction
        print(f"Starting BET extraction for {patient_id}")
        subprocess.run(['bet', b0_nii_path, 'bet_b0.nii', '-m', '-f', '0.25'])
        progress.update()  # Update progress after BET extraction

        # MNI registration
        print(f"Starting MNI registration for {patient_id}")
        subprocess.run(['mirtk', 'register', '-image', 'bet_b0.nii.gz', mni_path,
                        '-dofout', 'transfomation.txt', '-output', 'r_mni.nii'])
        progress.update()  # Update progress after MNI registration

        # Perform atlas registrations based on user input
        for atlas_type, do_register in [('network', register_network),
                                        ('visual', register_visual),
                                        ('motor', register_motor)]:
            if do_register:
                print(f"Starting {atlas_type} registration for {patient_id}")
                atlas_path = atlas_paths[atlas_type]
                output_path = f'r_{atlas_type}_ROI.nii'
                subprocess.run(['mirtk', 'transform-image', atlas_path, output_path,
                                '-target', 'bet_b0.nii.gz', '-dofin', 'transfomation.txt', '-interp', 'NN'])
                progress.update()  # Update progress after each atlas registration
                print(f'Registration complete for {patient_id}: {atlas_type}')

        progress.close()  

# List of patient folders
patient_folders = [
   patient_name,
   patient_name,
   patient_name,
   # continue adding patient names here...
]


# Call the registration function for all patients
mirtk_registration(patient_folders)
